In [28]:
import pandas as pd

In [29]:
df_gastro = pd.read_csv("merged_df_UmsatzTemperatur.csv")
df_dax = pd.read_csv("DAXHISTORIE.csv")

#Umwandlung des Jahr_Monat von 2023-1 zu 2023-01
df_gastro['Jahr_Monat'] = df_gastro['Jahr_Monat'].apply(lambda x: f"{x.split('-')[0]}-{int(x.split('-')[1]):02d}")


In [30]:
df_gastro

,Unnamed: 0,ID,Jahr,Monat,Gastro_Id,Umsatz in millionen,TMK,Jahr_Monat
0,0,0,1994,1,WZ08-55,92.3,4.2,1994-01
1,1,1,1994,1,WZ08-551,91.9,4.2,1994-01
2,2,2,1994,1,WZ08-552,97.3,4.2,1994-01
3,3,3,1994,1,WZ08-553,68.4,4.2,1994-01
4,4,4,1994,1,WZ08-559,127.1,4.2,1994-01
...,...,...,...,...,...,...,...,...
3724,3724,3955,2023,12,WZ08-561,94.3,6.5,2023-12
3725,3725,3956,2023,12,WZ08-562,96.5,6.5,2023-12
3726,3726,3957,2023,12,WZ08-563,70.1,6.5,2023-12
3727,3727,3958,2023,12,WZ08-561-01,91.9,6.5,2023-12


In [31]:
df_dax

,Date,Open,High,Low,Close,Adj Close,Volume
0,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0
1,1987-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,1988-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0
4,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0
...,...,...,...,...,...,...,...
9372,2024-05-28,18775.550781,18855.050781,18635.039063,18677.869141,18677.869141,52670400.0
9373,2024-05-29,18677.699219,18677.699219,18438.009766,18473.289063,18473.289063,59553600.0
9374,2024-05-30,18465.470703,18531.849609,18394.429688,18496.789063,18496.789063,52348300.0
9375,2024-05-31,18495.220703,18542.269531,18426.720703,18497.939453,18497.939453,144013800.0


In [32]:
df_dax['Date'] = pd.to_datetime(df_dax['Date'])

# Jahr und Monat extrahieren und als neue Spalte hinzufügen
df_dax['YearMonth'] = df_dax['Date'].dt.to_period('M')

# Gruppieren nach Jahr und Monat und aggregieren
aggregated_dax = df_dax.groupby('YearMonth').agg({
    'Open': 'first',
    'Close': 'last',
}).reset_index()

# Umbenennen der 'YearMonth' Spalte in 'Date' für Konsistenz
aggregated_dax['YearMonth'] = aggregated_dax['YearMonth'].astype(str)  # Umwandlung in String-Format 'YYYY-MM'
aggregated_dax.rename(columns={'YearMonth': 'Date'}, inplace=True)
aggregated_dax

,Date,Open,Close
0,1987-12,1005.190002,1005.190002
1,1988-01,956.489990,936.000000
2,1988-02,951.500000,1083.339966
3,1988-03,1087.319946,1062.329956
4,1988-04,1063.280029,1049.170044
...,...,...,...
434,2024-02,16834.130859,17678.189453
435,2024-03,17793.470703,18492.490234
436,2024-04,18492.359375,17932.169922
437,2024-05,17935.779297,18497.939453


In [33]:
min = df_gastro["Jahr_Monat"].min()
min

'1994-01'

In [34]:
max = df_gastro["Jahr_Monat"].max()
max

'2023-12'

In [35]:
# Step 1: Determine the date range from df_gastro
min_date = df_gastro["Jahr_Monat"].min()
max_date = df_gastro["Jahr_Monat"].max()

# Step 2: Filter the aggregated_dax DataFrame
filtered_dax = aggregated_dax[(aggregated_dax['Date'] >= min_date) & (aggregated_dax['Date'] <= max_date)]

# Step 3: Drop rows with any missing values
filtered_dax = filtered_dax.dropna()

# Display the filtered DataFrame
filtered_dax


,Date,Open,Close
73,1994-01,2252.429932,2192.600098
74,1994-02,2207.060059,2103.239990
75,1994-03,2101.989990,2142.879883
76,1994-04,2134.250000,2252.510010
77,1994-05,2261.229980,2137.340088
...,...,...,...
428,2023-08,16414.789063,15947.080078
429,2023-09,15936.339844,15386.580078
430,2023-10,15439.959961,14810.339844
431,2023-11,14851.959961,16215.429688


#### merge von aggregated_dax und gastro

In [36]:
merged_df_dax_gastro = pd.merge(df_gastro, filtered_dax, how='left', left_on='Jahr_Monat', right_on='Date')
merged_df_dax_gastro

,Unnamed: 0,ID,Jahr,Monat,Gastro_Id,Umsatz in millionen,TMK,Jahr_Monat,Date,Open,Close
0,0,0,1994,1,WZ08-55,92.3,4.2,1994-01,1994-01,2252.429932,2192.600098
1,1,1,1994,1,WZ08-551,91.9,4.2,1994-01,1994-01,2252.429932,2192.600098
2,2,2,1994,1,WZ08-552,97.3,4.2,1994-01,1994-01,2252.429932,2192.600098
3,3,3,1994,1,WZ08-553,68.4,4.2,1994-01,1994-01,2252.429932,2192.600098
4,4,4,1994,1,WZ08-559,127.1,4.2,1994-01,1994-01,2252.429932,2192.600098
...,...,...,...,...,...,...,...,...,...,...,...
3724,3724,3955,2023,12,WZ08-561,94.3,6.5,2023-12,2023-12,16296.919922,16751.640625
3725,3725,3956,2023,12,WZ08-562,96.5,6.5,2023-12,2023-12,16296.919922,16751.640625
3726,3726,3957,2023,12,WZ08-563,70.1,6.5,2023-12,2023-12,16296.919922,16751.640625
3727,3727,3958,2023,12,WZ08-561-01,91.9,6.5,2023-12,2023-12,16296.919922,16751.640625


In [37]:
merged_df_dax_gastro.drop(columns=["Unnamed: 0","ID","Jahr","Monat","Jahr_Monat"],inplace=True)
merged_df_dax_gastro

,Gastro_Id,Umsatz in millionen,TMK,Date,Open,Close
0,WZ08-55,92.3,4.2,1994-01,2252.429932,2192.600098
1,WZ08-551,91.9,4.2,1994-01,2252.429932,2192.600098
2,WZ08-552,97.3,4.2,1994-01,2252.429932,2192.600098
3,WZ08-553,68.4,4.2,1994-01,2252.429932,2192.600098
4,WZ08-559,127.1,4.2,1994-01,2252.429932,2192.600098
...,...,...,...,...,...,...
3724,WZ08-561,94.3,6.5,2023-12,16296.919922,16751.640625
3725,WZ08-562,96.5,6.5,2023-12,16296.919922,16751.640625
3726,WZ08-563,70.1,6.5,2023-12,16296.919922,16751.640625
3727,WZ08-561-01,91.9,6.5,2023-12,16296.919922,16751.640625


In [38]:
# Sample training data from original data
train_data = merged_df_dax_gastro.sample(frac=0.8, random_state=0)

# Get test data by dropping training data from original data
test_data = merged_df_dax_gastro.drop(train_data.index) 

print(f"Training data: {train_data.shape} \nTest data: {test_data.shape}")

Training data: (2983, 6) 
Test data: (746, 6)


In [39]:

# Features
X_series = train_data[[' TMK','Open','Close']]
X_df = train_data[[' TMK','Open','Close']]
y_series = train_data['Umsatz in millionen']

# Assign train data
X_train = X_df 
y_train = y_series


In [40]:
# Import
from sklearn.linear_model import LinearRegression

# Training
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)
print(f"Regression line: Y = {reg_all.intercept_:.4f} + {reg_all.coef_[0]:.4f} * TMK + {reg_all.coef_[1]:.4f} * Open + {reg_all.coef_[2]:.4f} * Close")

Regression line: Y = 125.4139 + 1.9376 * TMK + 0.0004 * Open + -0.0049 * Close


In [41]:
# Import
from sklearn.metrics import mean_squared_error

X_test = test_data[[' TMK','Open','Close']]
y_test = test_data['Umsatz in millionen']
predictions = reg_all.predict(X_test)
mse =  mean_squared_error(y_test, predictions)
print(f"Mean squared error (MSE) on test data: {mse:.2f}")

Mean squared error (MSE) on test data: 867.82


In [44]:
# Calculate Root Mean Squared Error (RMSE)
rmse = mse ** 0.5
print(f"Root Mean Squared Error (RMSE) on test data: {rmse:.2f}")

# Calculate the mean of the target variable in the test data
mean_target = y_test.mean()
print(f"Mean of target variable (Umsatz in millionen) in test data: {mean_target:.2f}")

# Calculate the relative error
relative_error = rmse / mean_target
print(f"Relative error: {relative_error:.2%}")


Root Mean Squared Error (RMSE) on test data: 29.46
Mean of target variable (Umsatz in millionen) in test data: 111.74
Relative error: 26.36%


In [53]:
# Example input data (temperature, DAX open, DAX close)
temperature = input('Temperatur')
dax_open = input('Dax Open')
dax_close = input('Dax close')

# Create a DataFrame with the input data
input_data = pd.DataFrame({
    ' TMK': [temperature],
    'Open': [dax_open],
    'Close': [dax_close]
})

# Function to predict Umsatz in millionen for each unique gastro ID
def predict_umsatz_for_gastro(model, input_data, merged_df_dax_gastro):
    predicted_results = []

    # Get unique gastro IDs
    unique_gastro_ids = merged_df_dax_gastro['Gastro_Id'].unique()

    # Iterate over each unique gastro ID
    for gastro_id in unique_gastro_ids:
        # Filter data for the current gastro ID
        data_for_gastro = merged_df_dax_gastro[merged_df_dax_gastro['Gastro_Id'] == gastro_id]
        
        # Prepare input data for prediction
        X_pred = input_data.copy()
        
        # Make predictions
        predictions = model.predict(X_pred)
        
        # Store predicted results
        predicted_results.append({
            'Gastro_Id': gastro_id,
            'Predicted_Umsatz_in_millionen': predictions[0]  # Assuming only one prediction per gastro ID
        })
    
    # Convert predicted results to DataFrame
    predicted_df = pd.DataFrame(predicted_results)
    
    return predicted_df

# Example usage
predicted_df = predict_umsatz_for_gastro(reg_all, input_data, merged_df_dax_gastro)

# Display predicted results
print(predicted_df)


      Gastro_Id  Predicted_Umsatz_in_millionen
0       WZ08-55                     167.007695
1      WZ08-551                     167.007695
2      WZ08-552                     167.007695
3      WZ08-553                     167.007695
4      WZ08-559                     167.007695
5       WZ08-56                     167.007695
6      WZ08-561                     167.007695
7      WZ08-562                     167.007695
8      WZ08-563                     167.007695
9   WZ08-561-01                     167.007695
10   WZ08-55-01                     167.007695
